In [22]:

from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek

# Python functions

In [4]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

# Tool calls

In [5]:
llm = ChatOpenAI(base_url = "https://api.deepseek.com/v1",model = "deepseek-chat")

In [13]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

message = llm_with_tools.invoke(query)
message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_0_bb3ae198-5c9e-464a-9648-201528cb03a3',
  'type': 'tool_call'}]

# Parsing

In [15]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
res = chain.invoke(query)